In [ ]:
# SAM 2のインストール
!git clone https://github.com/facebookresearch/segment-anything-2.git
%cd segment-anything-2
!pip install -e .


Cloning into 'segment-anything-2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 128.11 MiB | 13.88 MiB/s, done.
Resolving deltas: 100% (380/380), done.
/content/segment-anything-2
Obtaining file:///content/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [ ]:
# download models
# 他のモデルはこちら：https://github.com/facebookresearch/sam2?tab=readme-ov-file#model-description
# もしくはcheckpoints/download_ckpts.shから

!wget https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt -O checkpoints/sam2.1_hiera_large.pt

--2025-06-22 22:29:37--  https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.111, 13.226.210.15, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 898083611 (856M) [application/vnd.snesdev-page-table]
Saving to: ‘checkpoints/sam2.1_hiera_large.pt’

checkpoints/sam2.1_ 100%[===================>] 856.48M   140MB/s    in 6.0s    

2025-06-22 22:29:43 (143 MB/s) - ‘checkpoints/sam2.1_hiera_large.pt’ saved [898083611/898083611]



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import torch
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
import os
from PIL import Image, ImageDraw, ImageFont

class SAM2Segmentation:
    def __init__(self, model_cfg="sam2_hiera_l.yaml", checkpoint_path="sam2_hiera_large.pt"):
        """
        SAM2セグメンテーションクラス

        Args:
            model_cfg: モデル設定ファイルのパス
            checkpoint_path: チェックポイントファイルのパス
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"使用デバイス: {self.device}")

        # SAM2モデルの構築
        self.sam2_model = build_sam2(model_cfg, checkpoint_path, device=self.device)

        # 自動マスク生成器の初期化
        self.mask_generator = SAM2AutomaticMaskGenerator(
            model=self.sam2_model,
            points_per_side=32,
            pred_iou_thresh=0.7,
            stability_score_thresh=0.92,
            crop_n_layers=1,
            crop_n_points_downscale_factor=2,
            min_mask_region_area=100,
        )

    def load_image(self, image_path):
        """画像を読み込み、RGB形式で返す"""
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"画像を読み込めませんでした: {image_path}")
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    def generate_masks(self, image):
        """画像に対してセグメンテーションマスクを生成"""
        print("セグメンテーションマスクを生成中...")
        masks = self.mask_generator.generate(image)
        print(f"生成されたマスク数: {len(masks)}")
        return masks

    def draw_numbered_segments(self, image, masks, output_path="segmented_image.png", font_size=None):
        """
        セグメンテーション結果を可視化し、各領域に番号を印字

        Args:
            image: 元画像（RGB）
            masks: セグメンテーションマスクのリスト
            output_path: 出力画像のパス
            font_size: フォントサイズ（Noneの場合は自動調整）
        """
        # 結果画像の作成
        result_image = image.copy()

        # マスクを面積順にソート（大きい順）
        sorted_masks = sorted(masks, key=lambda x: x['area'], reverse=True)

        # カラーマップの作成
        colors = plt.cm.tab20(np.linspace(0, 1, len(sorted_masks)))

        # PIL画像に変換（テキスト描画のため）
        pil_image = Image.fromarray(result_image)
        draw = ImageDraw.Draw(pil_image)

        # フォントサイズの設定
        if font_size is None:
            font_size = max(20, min(image.shape[0], image.shape[1]) // 30)

        # フォントの取得を試行
        font = None
        font_candidates = [
            "arial.ttf",
            "Arial.ttf",
            "helvetica.ttf",
            "DejaVuSans.ttf",
            "NotoSans-Regular.ttf",
            "calibri.ttf",
            "Calibri.ttf"
        ]

        # システムフォントを順番に試行
        for font_name in font_candidates:
            try:
                font = ImageFont.truetype(font_name, font_size)
                break
            except (OSError, IOError):
                continue

        # システムフォントが見つからない場合はpillow-simsunを試行
        if font is None:
            try:
                import PIL.ImageFont
                # Pillowの内蔵フォントでサイズ指定可能なものを試行
                font = ImageFont.load_default()
                # load_default()は固定サイズなので、代替手段としてcv2で描画に変更
                use_cv2_text = True
            except:
                use_cv2_text = True
        else:
            use_cv2_text = False

        # 各マスクを処理
        for i, mask_info in enumerate(sorted_masks):
            mask = mask_info['segmentation']
            color = colors[i % len(colors)]

            # マスクの重ね合わせ（半透明）
            mask_colored = np.zeros_like(result_image)
            mask_colored[mask] = (np.array(color[:3]) * 255).astype(np.uint8)

            # アルファブレンド
            alpha = 0.3
            result_image = cv2.addWeighted(result_image, 1-alpha, mask_colored, alpha, 0)

            # マスクの重心を計算
            y_coords, x_coords = np.where(mask)
            if len(y_coords) > 0:
                centroid_x = int(np.mean(x_coords))
                centroid_y = int(np.mean(y_coords))

                # 番号を描画
                text = str(i + 1)

                if use_cv2_text:
                    # OpenCVでテキスト描画の場合は一時的にPIL画像をnumpy配列に変換
                    temp_array = np.array(pil_image)

                    # OpenCVでテキスト描画（フォントサイズをスケールで調整）
                    font_scale = font_size / 30.0  # 30を基準とした相対スケール
                    thickness = max(1, int(font_size / 15))  # 太さも調整

                    # テキストサイズを取得
                    (text_width, text_height), baseline = cv2.getTextSize(
                        text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness
                    )

                    # 背景円を描画
                    circle_radius = max(text_width, text_height) // 2 + int(font_size * 0.3)
                    cv2.circle(temp_array, (centroid_x, centroid_y), circle_radius, (255, 255, 255), -1)
                    cv2.circle(temp_array, (centroid_x, centroid_y), circle_radius, (0, 0, 0), 2)

                    # テキストの位置を調整
                    text_x = centroid_x - text_width // 2
                    text_y = centroid_y + text_height // 2

                    # 番号を描画
                    cv2.putText(temp_array, text, (text_x, text_y),
                               cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), thickness)

                    # PIL画像を更新
                    pil_image = Image.fromarray(temp_array)
                    draw = ImageDraw.Draw(pil_image)
                else:
                    # PILでテキスト描画（True Type フォント使用）
                    # テキストの境界ボックスを取得
                    bbox = draw.textbbox((0, 0), text, font=font)
                    text_width = bbox[2] - bbox[0]
                    text_height = bbox[3] - bbox[1]

                    # テキストの位置を調整
                    text_x = centroid_x - text_width // 2
                    text_y = centroid_y - text_height // 2

                    # 背景円を描画
                    circle_radius = max(text_width, text_height) // 2 + 5
                    draw.ellipse(
                        [(centroid_x - circle_radius, centroid_y - circle_radius),
                         (centroid_x + circle_radius, centroid_y + circle_radius)],
                        fill=(255, 255, 255, 200),
                        outline=(0, 0, 0, 255),
                        width=2
                    )

                    # 番号を描画
                    draw.text((text_x, text_y), text, fill=(0, 0, 0, 255), font=font)

        # 最終的にPIL画像をnumpy配列に変換
        result_image = np.array(pil_image)

        # 結果を保存
        cv2.imwrite(output_path, cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR))
        print(f"結果画像を保存しました: {output_path}")

        return result_image

    def create_mask_overlay(self, image, masks, output_path="mask_overlay.png"):
        """マスクオーバーレイ画像を作成"""
        overlay = np.zeros_like(image)

        for i, mask_info in enumerate(masks):
            mask = mask_info['segmentation']
            color = plt.cm.tab20(i / len(masks))[:3]
            overlay[mask] = (np.array(color) * 255).astype(np.uint8)

        # 元画像と重ね合わせ
        result = cv2.addWeighted(image, 0.7, overlay, 0.3, 0)
        cv2.imwrite(output_path, cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
        print(f"マスクオーバーレイ画像を保存しました: {output_path}")

        return result

    def save_mask_info(self, masks, output_path="mask_info.json"):
        """マスク情報をJSONファイルに保存"""
        import json

        # JSONに保存するデータを準備
        mask_data = {
            "total_segments": len(masks),
            "segments": []
        }

        for i, mask_info in enumerate(masks):
            segment_info = {
                "segment_id": i + 1,
                "area": int(mask_info['area']),
                "predicted_iou": float(mask_info['predicted_iou']),
                "stability_score": float(mask_info['stability_score']),
                "bbox": {
                    "x": int(mask_info['bbox'][0]),
                    "y": int(mask_info['bbox'][1]),
                    "width": int(mask_info['bbox'][2]),
                    "height": int(mask_info['bbox'][3])
                }
            }

            # 重心座標を計算
            mask = mask_info['segmentation']
            y_coords, x_coords = np.where(mask)
            if len(y_coords) > 0:
                centroid = {
                    "x": float(np.mean(x_coords)),
                    "y": float(np.mean(y_coords))
                }
                segment_info["centroid"] = centroid

            mask_data["segments"].append(segment_info)

        # JSONファイルに保存
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(mask_data, f, ensure_ascii=False, indent=2)

        print(f"マスク情報をJSONで保存しました: {output_path}")

    def process_image(self, image_path, output_dir="output", font_size=None):
        """
        画像を処理してセグメンテーション結果を出力

        Args:
            image_path: 入力画像のパス
            output_dir: 出力ディレクトリ
            font_size: 印字する数字のフォントサイズ（Noneの場合は自動調整）
        """
        # 出力ディレクトリの作成
        os.makedirs(output_dir, exist_ok=True)

        # 画像の読み込み
        image = self.load_image(image_path)
        print(f"画像サイズ: {image.shape}")

        # セグメンテーション実行
        masks = self.generate_masks(image)

        # 結果の可視化と保存
        base_name = os.path.splitext(os.path.basename(image_path))[0]

        # 番号付きセグメンテーション画像
        numbered_output = os.path.join(output_dir, f"{base_name}_numbered.png")
        self.draw_numbered_segments(image, masks, numbered_output, font_size=font_size)

        # マスクオーバーレイ画像
        overlay_output = os.path.join(output_dir, f"{base_name}_overlay.png")
        self.create_mask_overlay(image, masks, overlay_output)

        # マスク情報の保存（JSON形式）
        info_output = os.path.join(output_dir, f"{base_name}_info.json")
        self.save_mask_info(masks, info_output)

        return masks



In [ ]:
# 使用例

# SAM2セグメンテーションオブジェクトの作成
# 注意: 実際のモデルファイルのパスを指定してください
segmenter = SAM2Segmentation(
    model_cfg="configs/sam2.1/sam2.1_hiera_l.yaml",  # 設定ファイルのパス
    checkpoint_path="checkpoints/sam2.1_hiera_large.pt"  # チェックポイントのパス
)
# 画像のパスを指定
image_path = "/content/a3c3w3.jpg"  # 処理したい画像のパス

# セグメンテーション実行
try:
    masks = segmenter.process_image(image_path, output_dir="segmentation_results", font_size=20)  # フォントサイズを指定（例：40px）
    print(f"処理完了! {len(masks)}個の領域が検出されました。")

except Exception as e:
    print(f"エラーが発生しました: {e}")
    print("以下を確認してください:")
    print("1. SAM2モデルファイルが正しいパスに配置されているか")
    print("2. 入力画像が存在するか")
    print("3. 必要なライブラリがインストールされているか")

使用デバイス: cuda
画像サイズ: (1024, 1024, 3)
セグメンテーションマスクを生成中...


/content/segment-anything-2/sam2/sam2_image_predictor.py:431: UserWarning: /content/segment-anything-2/sam2/_C.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  masks = self._transforms.postprocess_masks(


生成されたマスク数: 21
結果画像を保存しました: segmentation_results/a3c3w3_numbered.png
マスクオーバーレイ画像を保存しました: segmentation_results/a3c3w3_overlay.png
マスク情報をJSONで保存しました: segmentation_results/a3c3w3_info.json
処理完了! 21個の領域が検出されました。


注意：初回実行時に以下の警告が出るのを確認しています。
おそらくはPyTorchのバージョンが適していないゆえの警告かと思います。今回は十分目的の動作はできているのでこのまま進めさせてください。後日原因が確認でき次第修正したいと思います。（2025.06.23 品川）

/content/segment-anything-2/sam2/sam2_image_predictor.py:431: UserWarning: /content/segment-anything-2/sam2/_C.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).